<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock_selection_paradigm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install pandas_ta --q

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
def calculate_tr(high, low, close):
    tr1 = high - low
    tr2 = abs(high - close.shift(1))
    tr3 = abs(low - close.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr

def calculate_dm(high, low):
    up_move = high - high.shift(1)
    down_move = low.shift(1) - low
    plus_dm = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)
    return plus_dm, minus_dm

def calculate_adx(high, low, close, period=14):
    tr = calculate_tr(high, low, close)
    plus_dm, minus_dm = calculate_dm(high, low)

    tr_smooth = tr.rolling(window=period).mean()
    plus_dm_smooth = plus_dm.rolling(window=period).mean()
    minus_dm_smooth = minus_dm.rolling(window=period).mean()

    plus_di = 100 * (plus_dm_smooth / tr_smooth)
    minus_di = 100 * (minus_dm_smooth / tr_smooth)

    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.rolling(window=period).mean()

    return adx


# Define the function to evaluate the stock
def evaluate_stock(ticker):
    # Set the time frames
    end_date = dt.datetime.now()
    start_date = end_date - dt.timedelta(days=365)  # 1 year of data

    # Download historical price data
    data = yf.download(ticker, start=start_date, end=end_date)

    # Check if data is available
    if data.empty:
        print(f"No data found for {ticker}.")
        return None

    # Calculate moving averages
    data['50_MA'] = data['Close'].rolling(window=50).mean()
    data['200_MA'] = data['Close'].rolling(window=200).mean()

    # Calculate RSI
    data['RSI'] = ta.rsi(data['Close'], length=14)

    # Calculate MACD
    macd = ta.macd(data['Close'])
    data['MACD'] = macd['MACD_12_26_9']
    data['MACD_Signal'] = macd['MACDs_12_26_9']
    '''
    # Calculate Bollinger Bands
    bollinger = ta.bbands(data['Close'])
    data['BB_upper'] = bollinger['BBU_20_2.0']
    data['BB_lower'] = bollinger['BBL_20_2.0']
    '''
    data['BB_upper'] = data['Close'].rolling(window=20).std() * 2 + data['Close']
    data['BB_lower'] = data['Close'].rolling(window=20).std() * 2 - data['Close']
    # Prepare the criteria dictionary
    criteria = {
        'Ticker': ticker,
        'Weekly Price Increase': False,
        'Monthly Price Increase': False,
        'Quarterly Price Increase': False,
        '50_MA above 200_MA (Golden Cross)': False,
        'Price above 50_MA': False,
        'Price above 200_MA': False,
        'RSI between 50 and 70': False,
        'Avg Volume Week > Month': False,
        'MACD above Signal Line': False,
        'Price above Upper Bollinger Band': False,
        'Higher Highs and Lows': False
    }

    # Evaluate criteria
    # 1. Price Performance
    try:
        weekly_price = data['Close'][-5]
        monthly_price = data['Close'][-21]
        quarterly_price = data['Close'][-63]
        current_price = data['Close'][-1]
    except IndexError:
        print(f"Not enough data for {ticker}.")
        return None

    # Weekly Performance
    if current_price > weekly_price:
        criteria['Weekly Price Increase'] = True

    # Monthly Performance
    if current_price > monthly_price:
        criteria['Monthly Price Increase'] = True

    # Quarterly Performance
    if current_price > quarterly_price:
        criteria['Quarterly Price Increase'] = True

    # 2. Moving Averages
    if data['50_MA'][-1] > data['200_MA'][-1]:
        criteria['50_MA above 200_MA (Golden Cross)'] = True

    if current_price > data['50_MA'][-1]:
        criteria['Price above 50_MA'] = True

    if current_price > data['200_MA'][-1]:
        criteria['Price above 200_MA'] = True

    # 3. RSI
    if data['RSI'][-1] > 50 and data['RSI'][-1] < 70:
        criteria['RSI between 50 and 70'] = True

    # 4. Trading Volume
    avg_volume_week = data['Volume'][-5:].mean()
    avg_volume_month = data['Volume'][-21:].mean()

    if avg_volume_week > avg_volume_month:
        criteria['Avg Volume Week > Month'] = True

    # 5. MACD
    if data['MACD'][-1] > data['MACD_Signal'][-1]:
        criteria['MACD above Signal Line'] = True

    # 6. Bollinger Bands
    if current_price > data['BB_upper'][-1]:
        criteria['Price above Upper Bollinger Band'] = True

    # 7. Higher Highs and Higher Lows
    recent_highs = data['High'][-20:]
    recent_lows = data['Low'][-20:]

    if recent_highs.is_monotonic_increasing and recent_lows.is_monotonic_increasing:
        criteria['Higher Highs and Lows'] = True

    return criteria


# Evaluate each stock and collect results
results = []

# List of stock tickers to evaluate
nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
nifty50 = pd.read_csv(nifty50_url)
stock_list = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()

for stock in stock_list:
    criteria = evaluate_stock(stock)
    if criteria:
        results.append(criteria)

# Convert the results into a pandas DataFrame
df_results = pd.DataFrame(results)

# Set the Ticker as the index
df_results.set_index('Ticker', inplace=True)




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
df_results

,Weekly Price Increase,Monthly Price Increase,Quarterly Price Increase,50_MA above 200_MA (Golden Cross),Price above 50_MA,Price above 200_MA,RSI between 50 and 70,Avg Volume Week > Month,MACD above Signal Line,Price above Upper Bollinger Band,Higher Highs and Lows
Ticker,,,,,,,,,,,
ADANIENT.NS,False,True,False,False,True,True,True,True,True,False,False
ADANIPORTS.NS,False,False,False,True,False,True,False,True,True,False,False
APOLLOHOSP.NS,False,False,True,True,True,True,False,True,False,False,False
ASIANPAINT.NS,False,False,True,True,False,True,False,False,False,False,False
AXISBANK.NS,False,False,False,True,False,True,False,True,False,False,False
BAJAJ-AUTO.NS,False,True,True,True,True,True,True,True,False,False,False
BAJFINANCE.NS,False,True,True,True,True,True,True,False,False,False,False
BAJAJFINSV.NS,False,True,True,True,True,True,True,True,False,False,False
BPCL.NS,True,False,True,True,True,True,True,True,True,False,False


In [ ]:
#Replace True with 1 and False with 0
df_numeric = df_results.replace({True: 1, False: 0})

# Sum across the rows
df_numeric['Row_Sum'] = df_numeric.sum(axis=1)


,Weekly Price Increase,Monthly Price Increase,Quarterly Price Increase,50_MA above 200_MA (Golden Cross),Price above 50_MA,Price above 200_MA,RSI between 50 and 70,Avg Volume Week > Month,MACD above Signal Line,Price above Upper Bollinger Band,Higher Highs and Lows,Row_Sum
Ticker,,,,,,,,,,,,
ADANIENT.NS,0,1,0,0,1,1,1,1,1,0,0,6
ADANIPORTS.NS,0,0,0,1,0,1,0,1,1,0,0,4
APOLLOHOSP.NS,0,0,1,1,1,1,0,1,0,0,0,5
ASIANPAINT.NS,0,0,1,1,0,1,0,0,0,0,0,3
AXISBANK.NS,0,0,0,1,0,1,0,1,0,0,0,3
BAJAJ-AUTO.NS,0,1,1,1,1,1,1,1,0,0,0,7
BAJFINANCE.NS,0,1,1,1,1,1,1,0,0,0,0,6
BAJAJFINSV.NS,0,1,1,1,1,1,1,1,0,0,0,7
BPCL.NS,1,0,1,1,1,1,1,1,1,0,0,8


In [ ]:
# Display the updated DataFrame
df_numeric.sort_values(by='Row_Sum', ascending=False)['Row_Sum']

,Row_Sum
Ticker,
BRITANNIA.NS,9
NTPC.NS,9
GRASIM.NS,9
HINDALCO.NS,9
SUNPHARMA.NS,9
BPCL.NS,8
M&M.NS,8
CIPLA.NS,8
TATASTEEL.NS,8
